Extracting and Visualizing stock data

In [84]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly
!pip install -U kaleido

In [85]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image, display

In [86]:
import plotly.io as pio
pio.renderers.default = "iframe_connected"

In [87]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Define Graphing Function

In [88]:
# def make_graph(stock_data, revenue_data, stock):
#     fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
#     stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
#     revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
#     fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
#     fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
#     fig.update_xaxes(title_text="Date", row=1, col=1)
#     fig.update_xaxes(title_text="Date", row=2, col=1)
#     fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
#     fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
#     fig.update_layout(showlegend=False,
#     height=900,
#     title=stock,
#     xaxis_rangeslider_visible=True)
#     fig.show()
#     from IPython.display import display, HTML
#     fig_html = fig.to_html()
#     display(HTML(fig_html))

#Above code doesnt save the graph images when notebook is downloaded, as display function doesnt show static images.Do try to check

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=("Historical Share Price", "Historical Revenue"),
        vertical_spacing=0.3
    )

    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']

    fig.add_trace(go.Scatter(
        x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True),
        y=stock_data_specific.Close.astype("float"),
        name="Share Price"
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True),
        y=revenue_data_specific.Revenue.astype("float"),
        name="Revenue"
    ), row=2, col=1)

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    fig.update_layout(
        showlegend=False,
        height=900,
        title=stock,
        xaxis_rangeslider_visible=True
    )

    # Display plot in notebook
    fig.show()


__Question 1 - Extracting Tesla Stock Data Using yfinance__

In [89]:
#Extract data using Ticker module and ticker symbol for Tesla is "TSLA"
tesla= yf.Ticker("TSLA")

In [90]:
#Extract stock info and store as a DataFrame using history function and pass max as period params
tesla_data=tesla.history(period="max")

In [91]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


__Question 2 - Extracting Tesla Revenue Data Using Webscraping__

In [92]:
html_data =requests.get("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm").text

In [93]:
#Parse the html data using beautiful_soup using parser i.e html5lib or html.parser
soup = BeautifulSoup(html_data, "html.parser")

In [94]:
#Using BeautifulSoup or the read_html function extract the table with Tesla Revenue and store it into a dataframe named tesla_revenue. The dataframe should have columns Date and Revenue
table_body= soup.find_all('tbody')[1]
#  Initialize empty dataframe
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
#  Loop through rows in the tbody
for row in table_body.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue:  # Only add rows with non-empty revenue
            tesla_revenue = pd.concat([tesla_revenue,pd.DataFrame([{"Date": date, "Revenue": revenue}])])
#: Display the dataframe
#print(tesla_revenue.head())
tesla_revenue.tail()

,Date,Revenue
0,2010-09-30,31
0,2010-06-30,28
0,2010-03-31,21
0,2009-09-30,46
0,2009-06-30,27


__Question 3 - Extracting GameStop Stock Data Using yfinance__

In [95]:
# Get gamestop Data
gamestop=yf.Ticker("GME")

In [96]:
##Extract stock info and store as a DataFrame using history function and pass max as period params
gamestop_data=gamestop.history(period="max")

In [97]:
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


__Question 4 - Extracting GameStop Revenue Data Using Webscraping__

Use the requests library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html. Save the text of the response as a variable named html_data_2.

In [98]:
html_data_2=requests.get(" https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html").text

In [99]:
#Parse the html data using beautiful_soup using parser i.e html5lib or html.parser.
soup1 = BeautifulSoup(html_data_2,"html.parser")

Using BeautifulSoup or the read_html function extract the table with GameStop Revenue and store it into a dataframe named gme_revenue. The dataframe should have columns Date and Revenue. Make sure the comma and dollar sign is removed from the Revenue column.

In [100]:
gme_revenue= pd.DataFrame(columns=['Data','Revenue'])
table2_body = soup1.find_all('tbody')[1]
for row in table2_body.find_all('tr'):
    col=row.find_all('td')
    if len(col)==2:
        date=col[0].text.strip()
        revenue= col[1].text.strip().replace("$", "").replace(",", "")
        if revenue:
            gme_revenue=pd.concat([gme_revenue,pd.DataFrame([{'Date':date,'Revenue':revenue}])])
print(gme_revenue.head())

  Data Revenue        Date
0  NaN    1021  2020-04-30
0  NaN    2194  2020-01-31
0  NaN    1439  2019-10-31
0  NaN    1286  2019-07-31
0  NaN    1548  2019-04-30


__Question 5 - Tesla Stock and Revenue Dashboard__

In [101]:
!pip install -U kaleido

In [102]:
make_graph(tesla_data,tesla_revenue,'Tesla')

/tmp/ipykernel_330/3217906817.py:34: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipykernel_330/3217906817.py:40: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



__Question 6 - GameStop Stock and Revenue Dashboard__

In [103]:
make_graph(gamestop_data, gme_revenue, 'Gamestop')

/tmp/ipykernel_330/3217906817.py:34: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipykernel_330/3217906817.py:40: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

